# Setup

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("/nfs/turbo/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl")
from travel import init_travel
init_travel()

import argparse
from collections import defaultdict, Counter
from copy import deepcopy
from itertools import product
import json
import numpy as np
import os
import pickle
from PIL import Image
from pprint import pprint
import shutil
import spacy
import time
import torch
from tqdm import tqdm
from transformers import AutoModelForVision2Seq, AutoModelForCausalLM, AutoProcessor, BitsAndBytesConfig, AutoModelForSequenceClassification, AutoTokenizer, PhrasalConstraint           

from travel.constants import RESULTS_DIR, IMAGES_CHUNK_SIZE, HF_TOKEN, CONFIG_PATH
from travel.data.captaincook4d import CaptainCook4DDataset
from travel.data.ego4d import Ego4DMistakeDetectionDataset
from travel.data.mistake_detection import MistakeDetectionTasks
from travel.data.vqa import VQAResponse, get_vqa_response_token_ids, VQAOutputs, DIALOG_START_TOKENS, IMAGE_TOKENS, USER_START_TOKENS, USER_END_TOKENS, ASSISTANT_START_TOKENS, ASSISTANT_END_TOKENS, IVQA_PREAMBLE, IVQA_SUCCESS_QUESTION
from travel.data.vqg import generate_vqg_prompt_icl
from travel.model import simple_lm_prompt_beam_search, simple_vlm_prompt_beam_search, compute_completion_log_likelihoods, compute_completion_log_likelihoods_encoder_decoder, compute_completion_log_likelihoods_vlm
from travel.model.metrics import question_coherence_metrics_nli, question_coherence_metrics_vlm, generate_det_curve, compile_accuracy_and_coherence_metrics, generate_3d_overview_graph
from travel.model.mistake_detection import MISTAKE_DETECTION_THRESHOLDS
from travel.model.nli import NLI_MODEL_PATH, NLI_BATCH_SIZE
from travel.model.vqa import run_vqa_with_visual_filter
from travel.model.vqg import cleanup_generated_question

/home/sstorks/.cache/pypoetry/virtualenvs/travel-nQET-zRt-py3.10/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Qualtrics API helpers:

In [ ]:
import requests
import json
import base64
import io
from PIL import Image
import uuid
from typing import List, Optional

class QualtricsAnnotationSurvey:
    def __init__(self, api_token: str, data_center: str):
        """
        Initialize Qualtrics API client
        
        Args:
            api_token: Your Qualtrics API token
            data_center: Your data center (e.g., 'ca1', 'fra1', 'sydney1')
        """
        self.api_token = api_token
        self.base_url = f"https://{data_center}.qualtrics.com/API/v3"
        self.headers = {
            "X-API-TOKEN": api_token,
            "Content-Type": "application/json"
        }
    
    def upload_image_to_library(self, pil_image: Image.Image, filename: str) -> Optional[str]:
        """
        Upload PIL Image to Qualtrics Graphics Library
        
        Args:
            pil_image: PIL Image object
            filename: Name for the uploaded file
            
        Returns:
            String URL of uploaded image or None if failed
        """
        # Convert PIL image to bytes
        img_bytes = io.BytesIO()
        pil_image.save(img_bytes, format='PNG')
        img_bytes.seek(0)
        
        # Upload to graphics library
        files = {
            'file': (filename, img_bytes, 'image/png')
        }
        
        headers_upload = {
            "X-API-TOKEN": self.api_token
        }
        
        url = f"{self.base_url}/libraries/GR_MAIN/graphics"
        
        try:
            response = requests.post(url, headers=headers_upload, files=files)
            response.raise_for_status()
            
            result = response.json()
            return result['result']['url']  # Return the URL of uploaded image
        except requests.exceptions.RequestException as e:
            print(f"Failed to upload image {filename}: {e}")
            return None
    
    def create_survey(self, survey_name: str) -> Optional[str]:
        """
        Create a new survey
        
        Args:
            survey_name: Name for the survey
            
        Returns:
            Survey ID or None if failed
        """
        data = {
            "SurveyName": survey_name,
            "Language": "EN",
            "ProjectCategory": "CORE"
        }
        
        url = f"{self.base_url}/survey-definitions"
        
        try:
            response = requests.post(url, headers=self.headers, json=data)
            response.raise_for_status()
            
            result = response.json()
            return result['result']['SurveyID']
        except requests.exceptions.RequestException as e:
            print(f"Failed to create survey: {e}")
            return None
    
    def create_mistake_detection_question(self, survey_id: str, image_url: str, 
                                        task_description: str, question_id: str) -> bool:
        """
        Create a mistake detection question with image and binary choice
        
        Args:
            survey_id: ID of the survey
            image_url: URL of the uploaded image
            task_description: Text describing the task
            question_id: Unique identifier for this question
            
        Returns:
            True if successful, False otherwise
        """
        # HTML content with image and task description
        question_html = f"""
        <div style="margin-bottom: 20px;">
            You will be shown pairs of procedural texts and egocentric (POV) photos from people's perspective after trying to perform the procedure. Based on the photo, you will be asked to determine whether the person has made a mistake in completing the given procedure, or successfully completed it. In making your decision, consider the following guidelines:
            <ul>
                <li>If you see the object that should be affected by the procedure and it appears to have undergone the action(s) in the procedure (e.g., moving to a specific location, slicing, opening), the image should be labeled a success. If the object's state contradicts the success state for the procedure, the image should be labeled a mistake.</li>                
                <li>If you don't see the object or only see part of it, it could either be a success or mistake. Make your best guess based on what you do see in the image.</li>
                <li>If you're ever not sure (e.g., the image doesn't have enough information or it's blurry/low quality), also make your best guess based on any context clues you see.</li>
            </ul>

            
            <h3>Target procedure:</h3>
            <p>{task_description}</p>
        </div>
        <div style="text-align: center; margin: 20px 0;">
            <img src="{image_url}" style="max-width: 100%; height: auto;" alt="Task completion image"/>
        </div>
        <p><strong>Question: Did the person make a mistake in completing the described task?</strong></p>
        """
        
        question_data = {
            "QuestionText": question_html,
            "DataExportTag": f"mistake_detection_{question_id}",
            "QuestionType": "MC",
            "Selector": "SAVR",  # Single Answer Vertical
            "Configuration": {
                "QuestionDescriptionOption": "UseText"
            },
            "QuestionDescription": f"Mistake Detection - {question_id}",
            "Choices": {
                "1": {
                    "Display": "Yes - There is a mistake"
                },
                "2": {
                    "Display": "No - Task completed successfully"
                }
            },
            "ChoiceOrder": ["1", "2"],
            "Validation": {
                "Settings": {
                    "ForceResponse": "ON",
                    "Type": "None"
                }
            }
        }
        
        url = f"{self.base_url}/survey-definitions/{survey_id}/questions"
        
        try:
            response = requests.post(url, headers=self.headers, json=question_data)
            response.raise_for_status()
            return True
        except requests.exceptions.RequestException as e:
            print(f"Failed to create question {question_id}: {e}")
            return False
    
    def create_confidence_question(self, survey_id: str, question_id: str) -> bool:
        """
        Create a confidence rating question (1-5 scale)
        
        Args:
            survey_id: ID of the survey
            question_id: Unique identifier for this question
            
        Returns:
            True if successful, False otherwise
        """
        question_data = {
            "QuestionText": "How confident are you in your answer?",
            "DataExportTag": f"confidence_{question_id}",
            "QuestionType": "MC",
            "Selector": "SAVR",
            "Configuration": {
                "QuestionDescriptionOption": "UseText"
            },
            "QuestionDescription": f"Confidence Rating - {question_id}",
            "Choices": {
                "1": {"Display": "1 - Not confident at all"},
                "2": {"Display": "2 - Slightly confident"},
                "3": {"Display": "3 - Moderately confident"},
                "4": {"Display": "4 - Very confident"},
                "5": {"Display": "5 - Extremely confident"}
            },
            "ChoiceOrder": ["1", "2", "3", "4", "5"],
            "Validation": {
                "Settings": {
                    "ForceResponse": "ON",
                    "Type": "None"
                }
            }
        }
        
        url = f"{self.base_url}/survey-definitions/{survey_id}/questions"
        
        try:
            response = requests.post(url, headers=self.headers, json=question_data)
            response.raise_for_status()
            return True
        except requests.exceptions.RequestException as e:
            print(f"Failed to create confidence question {question_id}: {e}")
            return False

def generate_annotation_survey(api_token: str, data_center: str, 
                             images: List[Image.Image], 
                             task_descriptions: List[str],
                             survey_name: str = "Mistake Detection Annotation") -> Optional[str]:
    """
    Generate a complete annotation survey with images and task descriptions
    
    Args:
        api_token: Qualtrics API token
        data_center: Qualtrics data center
        images: List of PIL Image objects
        task_descriptions: List of task description strings (same length as images)
        survey_name: Name for the created survey
        
    Returns:
        Survey ID if successful, None otherwise
    """
    if len(images) != len(task_descriptions):
        raise ValueError("Number of images must match number of task descriptions")
    
    # Initialize API client
    client = QualtricsAnnotationSurvey(api_token, data_center)
    
    # Create survey
    survey_id = client.create_survey(survey_name)
    if not survey_id:
        print("Failed to create survey")
        return None
    
    print(f"Created survey with ID: {survey_id}")
    
    # Process each image and description pair
    for i, (image, description) in enumerate(zip(images, task_descriptions)):
        question_id = f"q{i+1:03d}"
        
        # Upload image
        image_filename = f"annotation_image_{question_id}.png"
        image_url = client.upload_image_to_library(image, image_filename)
        
        if not image_url:
            print(f"Failed to upload image for question {question_id}")
            continue
        
        # Create mistake detection question
        if client.create_mistake_detection_question(survey_id, image_url, description, question_id):
            print(f"Created mistake detection question {question_id}")
        else:
            print(f"Failed to create mistake detection question {question_id}")
            continue
        
        # Create confidence question
        if client.create_confidence_question(survey_id, question_id):
            print(f"Created confidence question for {question_id}")
        else:
            print(f"Failed to create confidence question for {question_id}")
    
    print(f"Survey generation complete. Survey ID: {survey_id}")
    return survey_id

# Load dataset 

In [3]:
# Load approopriate evaluation dataset
dataset = None
for retry in range(5):
    print(f"Loading evaluation dataset (try {retry})...")
    try:
        dataset = Ego4DMistakeDetectionDataset(data_split="val", 
                                                mismatch_augmentation=True,
                                                multi_frame=False,
                                                debug_n_examples_per_class=250)
        break
    except Exception as e:
        print("Encountered error during data loading:")
        pprint(e)
        time.sleep(60)
if dataset is None:
    raise ValueError("Could not load dataset after retrying!")

Loading evaluation dataset (try 0)...


# Sample data

First reorganize data by mistake type (and success) to estimate distribution of labels.

In [4]:
from collections import defaultdict
from pprint import pprint

examples_by_mistake_type = defaultdict(list)

for example in dataset:
    examples_by_mistake_type[str(example.mistake_type)].append(example)

mistake_type_dist = {}
for mistake_type in examples_by_mistake_type:
    mistake_type_dist[mistake_type] = len(examples_by_mistake_type[mistake_type]) / len(dataset)

pprint(mistake_type_dist)

{'Action Incomplete': 0.102,
 'MisalignSRL_ARG1': 0.174,
 'MisalignSRL_V': 0.062,
 'MisalignSRL_V_ARG1': 0.162,
 'None': 0.5}


In [ ]:
import random
from PIL import Image

TOTAL_EXAMPLES_TO_ANNOTATE = 100
N_EXAMPLES_PER_ANNOTATOR = 25
examples_to_annotate = {
    "Action Incomplete": 10,
    "MisalignSRL_ARG1": 18,
    "MisalignSRL_V": 6,
    "MisalignSRL_V_ARG1": 16,
    "None": 50,
}
assert TOTAL_EXAMPLES_TO_ANNOTATE == sum(list(examples_to_annotate.values())), "Configured distribution of annotated example types doesn't add up to total number of expected examples to annotate."

selected_examples = []
for k in examples_to_annotate:
    selected_examples += random.sample(examples_by_mistake_type[k], examples_to_annotate[k])

assert len(selected_examples) == TOTAL_EXAMPLES_TO_ANNOTATE, "Didn't sample expected number of examples to annotate."

random.shuffle(selected_examples)
assert TOTAL_EXAMPLES_TO_ANNOTATE % N_EXAMPLES_PER_ANNOTATOR == 0, "Number of examples per annotator must evenly divide total number of annotated examples."
sample_chunks = []
# Split into even chunks
n_chunks = TOTAL_EXAMPLES_TO_ANNOTATE // N_EXAMPLES_PER_ANNOTATOR
for chunk in range(n_chunks):
    examples = selected_examples[N_EXAMPLES_PER_ANNOTATOR*chunk:N_EXAMPLES_PER_ANNOTATOR*(chunk+1)]
    assert all(len(e.frames) == 0 for e in examples)

    descriptions = [e.procedure_description for e in examples]
    images = [e.frames[0] for e in examples]
    
    assert len(descriptions) == len(images) == N_EXAMPLES_PER_ANNOTATOR, f"Didn't get correct number of descriptions or images for chunk {chunk}."

    sample_chunks.append((images, descriptions))

assert len(sample_chunks) == n_chunks, "Didn't get the correct number of chunks."

# Generate Qualtrics forms for annotation

In [ ]:
from travel.constants import QUALTRICS_API_TOKEN, QUALTRICS_DATA_CENTER

for (images, descriptions) in sample_chunks:
    # Generate survey
    survey_id = generate_annotation_survey(
        api_token=QUALTRICS_API_TOKEN,
        data_center=QUALTRICS_DATA_CENTER,
        images=images,
        task_descriptions=descriptions,
        survey_name="Procedural Mistake Detection Human Annotation"
    ) # NOTE: make sure questions are shuffled

    if survey_id:
        print(f"\n✅ Survey created successfully!")
        print(f"Survey ID: {survey_id}")
        print(f"You can access it at: https://{QUALTRICS_DATA_CENTER}.qualtrics.com/jfe/form/{survey_id}")
    else:
        print("❌ Survey creation failed")